<a href="https://colab.research.google.com/github/EladdadMahdi/EladdadMahdi/blob/main/01_methodologie_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src="https://datascientest.fr/train/assets/logo_datascientest.png" style="height:150px">

<hr style="border-width:2px;border-color:#75DFC1">
<h1 style = "text-align:center" >Méthodologie en Deep Learning </h1>
<h2 style = "text-align:center" > Exercice 1 - Détection simple d'un visage </h2></center>
<hr style="border-width:2px;border-color:#75DFC1">

> Ce notebook est destiné à pratiquer les notions évoquées dans le premier exercice du module sur une machine plus adaptée.
>
> Si c'est la première fois que vous utilisez colab, n'hésitez pas à jeter un coup d'oeil sur ce [notebook](https://colab.research.google.com/drive/1jXEKOk3mRYBqFWoVwJ0ZpsRJCWj46Yxt?usp=sharing).


* Exécuter la cellule suivante pour télécharger le jeu de données ainsi que le décompresser.

In [ ]:
!wget https://assets-datascientest.s3-eu-west-1.amazonaws.com/datasets/YouTube_Faces_light.zip
!unzip -q YouTube_Faces_light.zip

# Pour le jeu de données originel
# !wget https://assets-datascientest.s3-eu-west-1.amazonaws.com/datasets/YouTube_Faces_heavy.zip
# !unzip -q YouTube_Faces_heavy.zip

--2021-12-07 14:23:26--  https://assets-datascientest.s3.eu-west-1.amazonaws.com/datasets/words_light.zip
Resolving assets-datascientest.s3.eu-west-1.amazonaws.com (assets-datascientest.s3.eu-west-1.amazonaws.com)... 52.218.28.8
Connecting to assets-datascientest.s3.eu-west-1.amazonaws.com (assets-datascientest.s3.eu-west-1.amazonaws.com)|52.218.28.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 77465542 (74M) [application/zip]
Saving to: ‘words_light.zip’

words_light.zip     100%[===================>]  73.88M  74.9MB/s    in 1.0s    

2021-12-07 14:23:27 (74.9 MB/s) - ‘words_light.zip’ saved [77465542/77465542]



* Exécuter la cellule suivante pour vérifier que le GPU soit bien activé.

In [ ]:
import tensorflow as tf
if tf.test.gpu_device_name(): .
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please change your hardware accelerator")

> Le fichier *face_images.csv* ainsi que le dossier *images* maintenant présent dans le dossier racine du notebook.

* Charger le jeu de données **<i>'face_images.csv'</i>** dans le dataframe **`df`**.


* Faire un bref audit des données.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
df = pd.read_csv('face_images.csv')
df.head()

* Définir une fonction `load_image` avec comme argument **`filepath`** et **`resize`** qui charge, redimensionne et qui retourne l'image. **Utiliser uniquement des fonctions de `tensorflow`**.


* Charger et afficher la première image de **`df`**.


* Afficher ses dimensions.

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
folder_images = 'images/'

def load_image(filepath, resize=None):
    im = tf.io.read_file(folder_images + filepath)
    im = tf.image.decode_png(im, channels=3)
    if resize:
        return tf.image.resize(im, resize)
    else :
        return im


im = load_image(df.filename[0])
plt.imshow(im);

* Créer les colonnes suivantes à **`df`** : **<i>xmoy</i>**, **<i>ymoy</i>**, **<i>w</i>** et **<i>h</i>**.


* Afficher les premières lignes de **`df`**.

In [ ]:
# Insérer votre code ici



In [ ]:
#@title Solution
df['xmoy'] = (df.xmax + df.xmin)/2
df['ymoy'] = (df.ymax + df.ymin)/2
df['w'] = (df.xmax - df.xmin)
df['h'] = (df.ymax - df.ymin)
df.head()

* Exécuter la cellule suivante pour afficher la bounding box de la première image.

In [ ]:
import numpy as np

def show_bounding_box(im, bbox, normalised=True, color='r'):
    # Signification de bbox
    x, y, w, h = bbox
    # Convertir les cordonées (x,y,w,h) en (x1,x2,y1,y2)
    x1=x-w/2
    x2=x+w/2
    y1=y-h/2
    y2=y+h/2

    # Redimentionner en cas de normalisation
    if normalised:
        x1=x1*im.shape[1]
        x2=x2*im.shape[1]
        y1=y1*im.shape[0]
        y2=y2*im.shape[0]

    # Afficher l'image
    plt.imshow(im)

    # Afficher la bounding box
    plt.plot([x1,x2,x2,x1,x1],[y1,y1,y2,y2,y1],"r")

idx = 1
# Array de l'image
im = load_image(df.filename[idx])
# Coordonnées de la bounding box
bbox = df[['xmoy', 'ymoy', 'w', 'h']].values[idx]
# Afficher l'image ainsi que la bounding box
show_bounding_box(im, bbox)

* À l'aide des fonctions définies précédemment, charger une image et afficher sa bounding box.

In [ ]:
# Array de l'image
im = load_image(df.filename[8])
# Coordonnées de la bounding box
bbox = df[['xmoy', 'ymoy', 'w', 'h']].values[8]
# Afficher l'image ainsi que la bounding box
show_bounding_box(im, bbox, normalised=True)

* Exécuter la cellule suivante pour afficher aléatoirement des images avec leur bounding box.

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,5))
for j, i in enumerate(np.random.randint(0, len(df), size=[8])):
    plt.subplot(2,4,j+1)
    plt.axis('off')
    im = load_image(df.filename[i])
    bbox = df[['xmoy', 'ymoy', 'w', 'h']].values[i]
    show_bounding_box(im, bbox, normalised=True)

### Distribution des variables cibles.

* Afficher la distribution de `xmoy` et `ymoy`.


* Où se trouve généralement la bounding box dans l'image ?

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.displot(df.xmoy)
plt.show()
sns.displot(df.ymoy)
plt.show()
## La bounding box se trouve généralement au centre de l'image.

* Afficher la distribution de `w` et `h`.


* Est-ce que les petits objets sont bien représentés ?

In [ ]:
sns.displot(df.w)
plt.show()
sns.displot(df.h)
plt.show()
## Les objets ont généralement la même taille et les petits objets ne sont pas beaucoup réprésentés.

### Charger le jeu de données

* Séparer le jeu de données **`df.filepath`** et les variables cibles en un ensemble d'entraînement **`X_train_path`**, **`y_train`**, et en un ensemble de test **`X_test_path`**, **`y_test`**. Nous choisirons un rapport de 80% pour les données d'entraînements et une graine aléatoire 1234.


* Charger les images de **`X_test_path`** redimentionnées à [256,256,3]  en mémoire dans la variable **`X_test`**.

In [ ]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm

X_train_path, X_test_path, y_train, y_test = train_test_split(df.filename, df[['xmoy', 'ymoy', 'w', 'h']], train_size=0.8, random_state=1234)

X_test = []
for p in tqdm(X_test_path):
    im = load_image(p, (256,256)).numpy().astype(np.uint8)
    X_test.append(im)

X_test = np.array(X_test)

* Définir un dataset **`dataset_train`** de **`(X_train_path, y_train)`** à l'aide de la fonction `from_tensor_slices`.


* À l'aide de la méthode `map`, appliquer la fonction `load_image` à chaque valeur de **X_train_path**. Pour que le chargement s'éffectue en multi-thread, préciser l'argument `num_parallel_calls` égale à -1.


* Regrouper les observations sous forme de batch de taille 32.

In [ ]:
import tensorflow as tf

@tf.function
def load_image(filepath, resize=(256,256)):
    im = tf.io.read_file(folder_images+filepath)
    im = tf.image.decode_png(im, channels=3)
    return tf.image.resize(im, resize)

dataset_train = tf.data.Dataset.from_tensor_slices((X_train_path, y_train))

dataset_train = dataset_train.map(lambda x, y : [load_image(x), y], num_parallel_calls=-1).batch(32)

* Exécuter la cellule suivante pour comparer le temps de chargement entre une méthode single-threading et multi-threading.

In [ ]:
import time
import cv2
t0 = time.time()
for p in X_test_path.values[:32]:
    load_image(p)

print('Time to load 32 image in a single thread method :', time.time()-t0, 's')

iterator= iter(dataset_train.take(1))
t0 = time.time()
next(iterator)
print('Time for a multi-threading method :', time.time()-t0, 's')

### Définition du modèle de détection d'objet

* Charger le modèle `EfficientNetB0` de **`tensorflow.keras.applications`**.


* Freezer les poids du modèle.


* Afficher le résumé du modèle.

In [ ]:
from tensorflow.keras.applications import EfficientNetB0

# Load the model efficientNet
efficientNet = EfficientNetB0(include_top=False, input_shape=(256,256,3))

# Freeze the blackbone
for layer in efficientNet.layers:
    layer.trainable = False

### Partie régression

* Ajouter le modèle à un objet `Sequential` qui portera le nom de **`model`**.


* Ajouter à ce modèle une couche `GlobalAveragePooling2D`.


* Puis, ajouter quelques couches `Dense` et `Dropout`.


* Finir par une couche `Dense` avec 4 neurones et une fonction activation `'linear'`.

In [ ]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Conv2D, MaxPooling2D, BatchNormalization, LeakyReLU, Flatten
from tensorflow.keras.models import Model, Sequential, load_model

model = Sequential()
model.add(efficientNet)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='linear'))
model.summary()

### Définition de la fonction de perte

* Définir une fonction de perte `loss_function`  de votre choix avec comme argument **`y_true`** et **`y_pred`**. Attention à bien n'utiliser que des fonctions venant du package `tensorflow`.


* Compiler le modèle avec votre fonction de perte `loss_function` et avec un optimizer 'adam'.

In [ ]:
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

lambda_regression=10

def loss_function(y_true, y_pred):
    return lambda_regression*tf.reduce_mean(tf.square(y_true-y_pred), axis=-1)

# def loss_function(y_true, y_pred):
#     return lambda_coord*tf.reduce_mean(tf.square(y_true[...,:2]-y_pred[...,:2]), axis=-1) + lambda_largeur*tf.reduce_mean(tf.square(y_true[...,2:4]-y_pred[...,2:4]), axis=-1)


model.compile(loss=loss_function, optimizer=Adam(1e-3))

### Entraînement du modèle


* Entraîner le modèle à l'aide la méthode `fit_generator` sur 20 epochs.

In [ ]:
model.fit(dataset_train, epochs=20)

* Tester le modèle sur images de X_test en affichant la bounding box associée.

In [ ]:
import time
def show_img(img, model):
    plt.imshow(img)
    t0=time.time()
    x, y, w, h = model.predict(tf.expand_dims(img,0))[0]
    print(x, y, w, h)
    print("Execution time :",time.time()-t0,"secondes")
    show_bounding_box(img/255, [x,y,w,h])
    plt.show()

## Exemple :
show_img(X_test[3], model)

### Tester le modèle sur votre caméra

* Exécuter les deux prochaines cellules pour activer la webcam sur le notebook colab.

In [ ]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

def js_to_image(js_reply):
  image_bytes = b64decode(js_reply.split(',')[1])
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  img = cv2.imdecode(jpg_as_np, flags=1)
  return img

def bbox_to_bytes(bbox_array):
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  bbox_PIL.save(iobuf, format='png')
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))
  return bbox_bytes

def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
video_stream()
label_html = 'Capturing...'
bbox = ''
count = 0
labels_dict = {0 : 'No mask', 1 : 'Mask'}

while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break
    img = js_to_image(js_reply["img"])
    img_resized = cv2.resize(img, (256, 256))
    bbox_array = np.zeros([img.shape[0], img.shape[1], 4], dtype=np.uint8)
    x_m, y_m, w, h = model(tf.expand_dims(img_resized[:,:,[2,1,0]], 0))[0]
    x_m = (x_m * img.shape[1]).numpy()
    y_m = (y_m * img.shape[0]).numpy()
    w = (tf.abs(w) * img.shape[1]).numpy()
    h = (h * img.shape[0]).numpy()

    # Display
    bbox_array = cv2.rectangle(bbox_array, (int(x_m-w/2), int(y_m-h/2)), (int(x_m+w/2), int(y_m+h/2)), (255, 0, 0), 2)
    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255

    bbox_bytes = bbox_to_bytes(bbox_array)
    bbox = bbox_bytes